In [1]:
%load_ext cython

In [2]:
import numpy as np

In [3]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
cimport cython
from numpy import zeros, double as npy_double
from libc.math cimport sqrt

cdef void mean_sd_1d(const double[:] x, double* mean, double* std):
    cdef Py_ssize_t n = x.size, i

    cdef double k = x[0]
    cdef double Ex = 0., Ex2 = 0.
    mean[0] = 0.

    for i in range(n):
        mean[0] += x[i]
        Ex += x[i] - k
        Ex2 += (x[i] - k)**2
    
    std[0] = sqrt((Ex2 - (Ex**2 / n)) / (n - 1))
    mean[0] /= n
    
    
def Autocorrelation(const double[:, :, :] x, int lag, bint normalize):
    cdef Py_ssize_t m = x.shape[0], n = x.shape[1], p = x.shape[2], i, j, k
    cdef double mean1, mean2, std1, std2

    autocorr = zeros((m, p), dtype=npy_double)
    cdef double[:, :] ac = autocorr

    if normalize:
        for i in range(m):
            for k in range(p):
                mean_sd_1d(x[i, :n-lag, k], &mean1, &std1)
                mean_sd_1d(x[i, lag:, k], &mean2, &std2)

                for j in range(n-lag):
                    ac[i, k] += (x[i, j, k] - mean1) * (x[i, j+lag, k] - mean2)
                ac[i, k] /= (n - lag) * std1 * std2
    else:
        for i in range(m):
            for k in range(p):
                mean_sd_1d(x[i, :n-lag, k], &mean1, &std1)
                mean_sd_1d(x[i, lag:, k], &mean2, &std2)

                for j in range(n-lag):
                    ac[i, k] += x[i, j, k] * x[i, j+lag, k]
                ac[i, k] /= std1 * std2
    
    return autocorr

In [4]:
from autocorrelation import autocorrelation as fautocorr

In [10]:
x = np.random.rand(50000, 150, 3)
xf = np.asfortranarray(x.transpose([1, 2, 0]))

In [12]:
%timeit Autocorrelation(x, 1, True)

137 ms ± 9.66 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit fautocorr(xf, 1, True)

54.8 ms ± 9.49 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
np.allclose(fautocorr(xf, 1, True).T, Autocorrelation(x, 1, True))

True